In [4]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import os
import gc

In [8]:
def partition(array, begin, end):
    pivot = begin
    for i in range(begin+1, end+1):
        if array[i][1] >= array[begin][1]:
            pivot += 1
            array[i], array[pivot] = array[pivot], array[i]
    array[pivot], array[begin] = array[begin], array[pivot]
    return pivot



def quicksort(array, begin=0, end=None):
    if end is None:
        end = len(array) - 1
    def _quicksort(array, begin, end):
        if begin >= end:
            return
        pivot = partition(array, begin, end)
        _quicksort(array, begin, pivot-1)
        _quicksort(array, pivot+1, end)
    return _quicksort(array, begin, end)

In [10]:
TargetClass='Pvalue'
directory='/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/Batches/censored_CTNNB1_Target'
if not os.path.exists(directory):
    print("Bad routing.")

for filename in os.listdir(directory):
    data = pd.read_csv(directory+'/'+filename)
    data['Pvalue']=list(map(lambda x: 1 if x<=0.01 else (0 if x>=0.5 else -1),data['Pvalue']))
    data.pop('Unnamed: 0')
    data.pop('TrueIndex')
    data.pop('CauseGene')
    data.pop('EffectGene')
    data.pop('Replicate')
    data.pop('Treatment')
    preliminary_results=[]
    for key in data:
        if key!=TargetClass:
            result=ttest_ind(data[TargetClass],data[key])
            preliminary_results.append([key,result[1]])
    quicksort(preliminary_results)
    # write out
    df = pd.DataFrame(preliminary_results)
    df.to_csv("/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/FeatureSelection/T-test/censored_CTNNB1_Target/results_"+filename)
    del df
    del data
    gc.collect()